# The Sound of Hydrogen

*adapted from [https://github.com/Carreau/posts/blob/master/07-the-sound-of-hydrogen.ipynb](https://github.com/Carreau/posts/blob/master/07-the-sound-of-hydrogen.ipynb)* 

Inspired by [minutephysics](https://www.youtube.com/watch?v=qyi5SvPlMXc), and the explanation of how to do it in mathematica: [The sound of hydrogen](https://www.youtube.com/watch?v=IhvW8yZdE5A).

The goal of this notebook is to show how one can play a sound file in a notebook using Html5 `<audio>` tag to play it directly inside the browser.

To do this we use the spectrum of hydrogen that we shift the into the audible range. You can listen to it in the last cell of this notebook.
Wait a few seconds if you are on nbviewer, the notebook is not light (someone to update it to use mp3? ogg? other compressed format?)

Please be aware that the html5 player is not working on some old browser and IE.

## HTML5 WAV player

this is a wrapper that takes numpy array and publishes an html `<audio>` tag to listen to it

In [ ]:
from __future__ import division
import StringIO
import base64
import struct
from scipy.io import wavfile
from IPython.core.display import HTML

def wavPlayer(data, rate):
    """ will display html 5 player for compatible browser

    The browser needs to know how to play wav through html5.

    There is no autoplay to prevent file playing when the browser opens
    
    Adapted from SciPy.io.
    """
    
    buffer = StringIO.StringIO()
    buffer.write(b'RIFF')
    buffer.write(b'\x00\x00\x00\x00')
    buffer.write(b'WAVE')

    buffer.write(b'fmt ')
    if data.ndim == 1:
        noc = 1
    else:
        noc = data.shape[1]
    bits = data.dtype.itemsize * 8
    sbytes = rate*(bits // 8)*noc
    ba = noc * (bits // 8)
    buffer.write(struct.pack('<ihHIIHH', 16, 1, noc, rate, sbytes, ba, bits))

    # data chunk
    buffer.write(b'data')
    buffer.write(struct.pack('<i', data.nbytes))

    if data.dtype.byteorder == '>' or (data.dtype.byteorder == '=' and sys.byteorder == 'big'):
        data = data.byteswap()

    buffer.write(data.tostring())
    #return buffer.getvalue()
    
    # Determine file size and place it in correct
    #  position at start of the file.
    size = buffer.tell()
    buffer.seek(4)
    buffer.write(struct.pack('<i', size-8))
    
    val = buffer.getvalue()
    
    #src = """
    #<head>
    #<meta http-equiv="Content-Type" content="text/html; charset=utf-8">
    #<title>Simple Test</title>
    #</head>
    src = """
    <body>
    <audio controls="controls" style="width:600px" >
      <source controls src="data:audio/wav;base64,{base64}" type="audio/wav" />
      Your browser does not support the audio element.
    </audio>
    </body>
    """.format(base64=base64.encodestring(val))
    display(HTML(src))

### Test the wav Player

let's try to first just play an A (440 Hz).

In [ ]:
%pylab inline

In [ ]:
from __future__ import division, print_function

from scipy import vectorize
import scipy.constants as const

some constants for our audio file

In [ ]:
rate = 44100  # in Hz
duration = 5  # in sec
time = np.linspace(0, duration, num=rate*duration)

this will give us sin with the right amplitude to use with wav files

In [ ]:
normedsin = lambda f,t : 2**12*sin(2*pi*f*t)

define A as a 440 Hz sin function 

In [ ]:
la    = lambda t : normedsin(440,t)

plot the first 25 ms and show full series in a Html 5 audio player

In [ ]:
plot(time[0:1000], la(time)[0:1000])
ampl = la(time).astype(np.int16)

wavPlayer(ampl, rate)

## Hydrogen Spectrum

The different frequencies emmited by an hydrogen atom are given by the Rydberg formula:

$$ {1 \over \lambda}  = R \left({1\over n_1}-{1\over n_2}\right) $$

Which gives a similar relation on the emitted frequencies of the Hydrogen :

$$ f_{n,m}={c \over \lambda}  = {R_h\over h} \left({1\over n}-{1\over m}\right) $$

for $n=1$ we've got the Lyman series, and for $n=2$ we have the Balmer series

fundamental frequency of hydrogen

In [ ]:
f0 = const.Rydberg*const.c
fshift = 440
print("The highest frequency of hydrogen is ",f0,"Hz and correspond to n = 1, m = ∞")
print("we can shift the spectrum for it to be at 440 Hz (A)")

In [ ]:
ryd = lambda n,m : fshift*(1/(n**2) -1/(m**2))
flyman = lambda x : ryd(1,x)
fbalmer = lambda x : ryd(2,x)

define the sum and a vectorized function to work on a by element basis

In [ ]:
ser = lambda t : sum( [normedsin(flyman(i),t)+normedsin(fbalmer(i+1),t) for i in range(2,8)])
serv = vectorize(ser)

In [ ]:
ss = serv(time)

In [ ]:
plot(time, ss)
ssnorm = 2**15*ss/ ss.max()
wavPlayer(ssnorm.astype(np.int16), rate)